In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
from sklearn.datasets import load_digits
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import euclidean_distances
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.feature_selection import VarianceThreshold

In [2]:
# loading the digits data set.
digits = load_digits()

print ("We have", digits.data.shape[0], "data points in total.")
print ("Each data point has", digits.data.shape[1], "features. In fact, each instance is an 8x8 picture of a digit.")
print ("Each digit's target/class/label is one of", list(set(digits.target)))

# We want to turn the problem into a binary classification one.
# Therefore, from the data set, we only pick those digits whose label is either 0 or 1
X = digits.data[(digits.target==0)|(digits.target==1)]
Y = digits.target[(digits.target==0)|(digits.target==1)]
print ("The reamining number of data points is", X.shape[0])
print(X[0:30])
print(Y[0:30])
print(X.shape)


We have 1797 data points in total.
Each data point has 64 features. In fact, each instance is an 8x8 picture of a digit.
Each digit's target/class/label is one of [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
The reamining number of data points is 360
[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  1. ...  3.  0.  0.]
 ...
 [ 0.  0.  0. ... 14.  0.  0.]
 [ 0.  0. 12. ...  2.  0.  0.]
 [ 0.  0.  0. ... 12.  0.  0.]]
[0 1 0 1 0 1 0 0 1 1 0 0 0 1 1 0 0 0 1 1 1 1 1 0 1 0 0 1 0 1]
(360, 64)


In [3]:
def show_some_digits(X, Y):

    # Here we just want to visualize some of the data points in the data set, along with their labels. 
    #
    # Create a large figure, to be filled with multiple subplots
    plt.figure(figsize=(20,8))
    # Go over the first 30 data points
    for index, (image, label) in enumerate(zip(X[0:30], Y[0:30])):
        plt.subplot(3, 10, index + 1)
        # Reshape the 64 features of each data point into an 8x8 image and then plot it.
        plt.imshow(np.reshape(image, (8,8)), cmap=plt.cm.gray)
        # Write the label of each point.
        plt.title('Target Label: %i\n' % label, fontsize = 10)
    return


# Here we create a noisy version of the data set
# The way that we do it is we go over all the pixels of
# each of the data points; then with probability p we multiply
# the value of that pixel by 0 (makring it essentially black).
# Otherwise (with probability 1-p) we multiply the value of that
# pixel by 1 (essentially keeping the pixel untouched)

# This the probability p of dropping a pixel (making it zero)
def make_it_noisy (X, drop_probability):
# This is how we create the noisy data set
    return np.multiply(X, np.random.choice([0, 1], size=(360,64), p=[drop_probability, 1 - drop_probability]))


print ("Hear are some of the original digits (first three rows)...and then their noisy versions")
# show_some_digits(X, Y)
X_noisy = make_it_noisy (X, 0.6)
# show_some_digits(X_noisy, Y)


Hear are some of the original digits (first three rows)...and then their noisy versions


In [4]:
# Here we define our own nearest neighbor classifier
# Any classifier should implement two main functions.
# The first one is fit() which takes the training data
# and does the training. The second one is predict()
# which is used after the training to predict the label
# of new points.
class MyNNClassifier(BaseEstimator, ClassifierMixin):

    def __init__(self, demo_param='demo'):
        self.demo_param = demo_param
        return

    # For the case of nearest neighbor , we don't really have any training;
    # we just need to store the training data points.
    def fit(self, X, Y):
        # Check that X and y have correct shape
        X, Y = check_X_y(X, Y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(Y)
        self.X_ = X
        self.Y_ = Y

        # Return the classifier
        return self

    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self, ['X_', 'Y_'])
        # Input validation
        X = check_array(X)
        
        # This is the main step of NN which is finding the closest point in the training data
        closest = np.argmin(euclidean_distances(X, self.X_), axis=1)
        return self.Y_[closest]



In [5]:
# This is our k-nearst neighbor classifer
class MyKnearstNeighbor(BaseEstimator, ClassifierMixin):
    
    def __init__(self, demo_param='demo'):
        self.demo_param = demo_param
        return 
    
    def fit(self, X, Y):
        # Check that X and y have correct shape
        X, Y = check_X_y(X, Y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(Y)
        self.X_ = X
        self.Y_ = Y
    
        self.k = 11
        return self
    
    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self, ['X_', 'Y_'])
        # Input validation
        X = check_array(X)
    
        Y = []
        for x in X:
            dist = np.argpartition([i[0] for i in euclidean_distances(self.X_, [x])], self.k)[:self.k]
            # Count number of occurrences of each value 
            bincount = np.bincount([self.Y_[i] for i in dist])
            # get the max index
            Y.insert(len(Y), np.argmax(bincount))
        return Y

In [6]:
# This is our soft-margin SVM classifer.
class MySoftSVMClassifier(BaseEstimator, ClassifierMixin):

    def __init__(self, demo_param='demo'):
        self.demo_param = demo_param
        return

    def fit(self, X, Y):
        # Check that X and y have correct shape
        X, Y = check_X_y(X, Y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(Y)
        self.X_ = X
        self.Y_ = Y
        
        # We use Stochastic Gradient Descent (SGD) to train a soft-margin SVM
        # The loss function is therefore the hinge loss and
        # ... the regularizaiton term will be l2 (Euclidean)
        # The constant multiplied by the regulariztion term is alpha (in the class we used lambda)
        # max_iter is the maximum number of iterations for the optimization
        self.softSVM = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000, alpha=.1)
        self.softSVM.fit(X, Y)

        # Return the classifier
        return self

    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self, ['X_', 'Y_'])
        # Input validation
        X = check_array(X)
        
        return self.softSVM.predict(X)



In [7]:
# This is our SVC 
class MyKernelVMClassifier(BaseEstimator, ClassifierMixin):
    
    def __init__(self, demo_param='demo'):
        self.demo_param = demo_param
        return

    def fit(self, X, Y):
        # Check that X and y have correct shape
        X, Y = check_X_y(X, Y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(Y)
        self.X_ = X
        self.Y_ = Y
        
        # the recommand C values: 10, 100, 1000
        self.clf = SVC(kernel='rbf', gamma='scale', max_iter=1000, C=1000)
        self.clf.fit(X, Y)
        return self
        

    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self, ['X_', 'Y_'])
        # Input validation
        X = check_array(X)
        return self.clf.predict(X)

In [8]:
# Bonus Question
class MyBonusQuestion(BaseEstimator, ClassifierMixin):

    def __init__(self, demo_param='demo'):
        self.demo_param = demo_param
        return

    def fit(self, X, Y):
        # Check that X and y have correct shape
        X, Y = check_X_y(X, Y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(Y)
        self.X_ = X
        self.Y_ = Y
        
        self.selector = SelectPercentile(f_classif, percentile=10)
        self.selector.fit(X, Y)
        self.clif = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
        self.clif.fit(self.selector.transform(X), Y)
        # Return the classifier
        return self

    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self, ['X_', 'Y_'])
        # Input validation
        X = check_array(X)
        
        return self.clif.predict(self.selector.transform(X))

In [9]:
# We want to test the accuracy of SVM, Nearest Neighbor and
# ... Logistic Regression classifiers

# Our own NN classifier
NN = MyNNClassifier()
# Our own Soft SVM classifier
softSVM = MySoftSVMClassifier()
# Logistic Regression directly from scikit library
LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
# K-nearest-neighbor classifier 
kNN = MyKnearstNeighbor()
# Our own SVC classifier
kernal_version = MyKernelVMClassifier()
# Bonus
improve_version = MyBonusQuestion()

softSVM_scores_all = []
NN_scores_all = []
LR_scores_all = []
kNN_scores_all = []
SVC_scores_all = []
IV_scores_all = []

# We use cross-validation to determine the accuracy.
# In t-fold cross-validation, the data is divided into
# t subsets. One of the subsets is used for testing, and 
# the other t-1 subsets (folds) are used for training.
# The test score is then calculated. Now, we could have 
# selected the test subset to be one of the other folds;
# therefore, we can have t different possible ways of 
# computing the socres. We take the average of these t
# possible ways as a measure for accuracy of the method.

for t in range(2,10):
    softSVM_scores = cross_val_score(softSVM, X_noisy, Y, cv=t)
    softSVM_scores_all.append(softSVM_scores.mean())    
    NN_scores = cross_val_score(NN, X_noisy, Y, cv=t)
    NN_scores_all.append(NN_scores.mean())
    LR_scores = cross_val_score(LR, X_noisy, Y, cv=t)
    LR_scores_all.append(LR_scores.mean())
    kNN_scores = cross_val_score(kNN, X_noisy, Y, cv=t)
    kNN_scores_all.append(kNN_scores.mean())
    SVC_scores = cross_val_score(kernal_version, X_noisy, Y, cv=t)
    SVC_scores_all.append(SVC_scores.mean())
    IV_scores = cross_val_score(improve_version, X_noisy, Y, cv=t)
    IV_scores_all.append(IV_scores.mean())
    

# Here we plot the cross-validation accuracy of NN and SVM 
# ... for different number of folds

plt.figure(figsize=(12,8))
plt.plot(softSVM_scores_all)
plt.plot(NN_scores_all)
plt.plot(LR_scores_all)
# plt.plot(kNN_scores_all)
# plt.plot(SVC_scores_all)
plt.plot(IV_scores_all)
# 'K-nearest-neighbor', 'SVC'
plt.legend(['Soft SVM', 'Nearest Neighbor', 'Logistic Regression', 'Bonus'])
plt.title('Accuracy of SVM vs NN vs Logistic Regression vs Bonus for different #folds')
plt.xlabel('Number of folds')
plt.ylabel('Accuracy')

plt.show()

D:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning

D:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [ 0  7  8 15 23 31 32 39 40 47 48 56] are constant.
  UserWarning)
D:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
D:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [ 0  7  8 15 16 23 24 31 32 39 40 47 48 56 57] are constant.
  UserWarning)
D:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
D:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but 

D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


D:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [ 0  7  8 15 23 31 32 39 40 47 48 56] are constant.
  UserWarning)
D:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
D:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [ 0  7  8 15 23 31 32 39 40 47 48 56] are constant.
  UserWarning)
D:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
D:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [ 0  7  8 15 16 23 24 31 32 39 40 47 48 56 57] are constant.
  UserWarning)
D:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f =

D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


...

D:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but 

D:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning

D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


D:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [ 0  7  8 15 23 31 32 39 40 47 48 56] are constant.
  UserWarning)
D:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
D:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [ 0  7  8 15 23 31 32 39 40 47 48 56] are constant.
  UserWarning)
D:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
D:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [ 0  7  8 15 23 31 32 39 40 47 48 56] are constant.
  UserWarning)
D:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / ms